In [19]:
import numpy as np
from rasterian.trasformation.stack import create_stacked_raster
from rasterian.trasformation.scale_intensity_correction import equalize_adapthist
from rasterian.focal.global_convolution import conv_media, conv_matrix, conv_image

### Create a raster stacked numpy narray

In [4]:
# change path to your raster bands folder
bands_files_path_dir = "../landsat_8_2022/"

# stack bands to a multiband raster
stacked_raster, raster_meta = create_stacked_raster(bands_files_path_dir)

['../landsat_8_2022/landsat_8_2022_b1.tif', '../landsat_8_2022/landsat_8_2022_b2.tif', '../landsat_8_2022/landsat_8_2022_b3.tif', '../landsat_8_2022/landsat_8_2022_b4.tif', '../landsat_8_2022/landsat_8_2022_b5.tif', '../landsat_8_2022/landsat_8_2022_b6.tif', '../landsat_8_2022/landsat_8_2022_b7.tif']


In [ ]:
# rescale raster
stacked_raster = equalize_adapthist(stacked_raster)

### Calculate media of pixels
The function calculates the media of the pixel values using a moving window of nxn.      
If n==3, is the same than using the function conv_matrix, with a matrix == [[1,1,1], [1,1,1], [1,1,1]]

In [12]:
# calculate the convolution for a band
blue_band_media = conv_media(stacked_raster[0], 3)

In [8]:
# calculate the convolution for the whole raster
stacked_raster_media = conv_image(stacked_raster, 3)

In [13]:
# check results
print(f"Blue mean value: {stacked_raster[0].mean()}")
print(f"Blue convolution mean value: {blue_band_media.mean()}")
print(f"Blue convolution mean value: {stacked_raster_media[0].mean()}")

Blue mean value: 6865.35550445148
Blue convolution mean value: 6865.355504451471
Blue convolution mean value: 6865.355504451471


### Calculate convolution of pixels using a matrix
The function calculates the media of the pixel values using a moving window of weights matrix (convolution kernel).      

In [9]:
# calculate the convolution for a band
convolution_kernel = [[3,2,1], [2,2,2], [3,0,1]]
blue_band_conv = conv_matrix(stacked_raster[0], convolution_kernel)

In [10]:
# calculate the convolution for the whole raster
convolution_kernel = [[3,2,1], [2,2,2], [3,0,1]]
stacked_raster_conv = conv_image(stacked_raster, convolution_kernel)

In [14]:
print(f"Blue mean value: {stacked_raster[0].mean()}")
print(f"Blue convolution mean value: {blue_band_conv.mean()}")
print(f"Blue convolution mean value: {stacked_raster_conv[0].mean()}")

Blue mean value: 6865.35550445148
Blue convolution mean value: 12205.076452358218
Blue convolution mean value: 12205.076452358218


### Low-Pass Filter (bluring)
Each pixel is computed as the average of the surrounding pixels.

In [16]:
# to applya convolution kernel of weights 1/9, pass a matrix of 1s.

convolution_kernel = [[1,1,1], [1,1,1], [1,1,1]]
stacked_raster_lowpass = conv_matrix(stacked_raster[0], convolution_kernel)

In [17]:
# If we didn't want to filter so harshly, we could change the kernel to reduce the averaging, for example:

convolution_kernel = [[0,1,0], [1,4,1], [0,1,0]]
stacked_raster_lowpass = conv_matrix(stacked_raster[0], convolution_kernel)

In [18]:
# Gaussian Blur - As you can see, it's a weighted mean of the surrounding pixels that gives more weight to the pixel near the current pixel. 

convolution_kernel = [
    [1,4,6,4,1],
    [4,16,24,16,4],
    [6,24,36,24,6],
    [4,16,24,16,4],
    [1,4,6,4,1]
]
stacked_raster_lowpass = conv_matrix(stacked_raster[0], convolution_kernel)

### High-Pass Filter (sharpening)
In this kernel, the pixel is boosted when the neighbor pixels are different.

In [ ]:
convolution_kernel = [[0,-0.5,0], [-0.5,3,-0.5], [0,-0.5,0]]
stacked_raster_highpass = conv_matrix(stacked_raster[0], convolution_kernel)

### Edge detection
There are multiple ways to do edge detection. We will present the Sobel Operator here.

In [ ]:
convolution_kernel = [[-1,0,1], [-2,0,2], [-1,0,1]]
stacked_raster_edge_x = conv_matrix(stacked_raster[0], convolution_kernel)

In [ ]:
convolution_kernel = [[-1,-2,-1], [0,0,0], [1,2,1]]
stacked_raster_edge_y = conv_matrix(stacked_raster[0], convolution_kernel)

In [ ]:
# We compute the convolution between the image (converted in black and white) and the two kernels separately. 
# That gives us, for each pixel, the values magx and magy. The value of the current pixel is set at rtsq( magx² + magy² )
raster_edges = np.sqrt(stacked_raster_edge_x **2 + stacked_raster_edge_y **2)